<a href="https://colab.research.google.com/github/gtoubian/cce/blob/main/4_5_Machine_Learning_Modelling_Lecture.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#4.4 Modelling

In today's lecture, we will be going over an example of using a Machine Learning Model to run a predictive analysis. We will be working with the linnerud dataset from sklearn and we are going to create a model that will try to predict someone's waist size from information on the number of chinups, Situps and Jumping Jacks  they are able to perform in a fixed time period.

In [2]:
from sklearn.datasets import load_linnerud
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

In [8]:
data = load_linnerud()
df = pd.DataFrame(data.data, columns=data.feature_names)
df.head()

,Chins,Situps,Jumps
0,5.0,162.0,60.0
1,2.0,110.0,60.0
2,12.0,101.0,101.0
3,12.0,105.0,37.0
4,13.0,155.0,58.0


When creating a model, we want to split our data into Training and Test Sets. The training set is essentially the set of data we use to calculate our model parameters using Ordinary Least Squares (OLS) and our test set is the set of data we use to see how well our model holds up given new data that it hasn't seen before.

In [6]:
y = data.target
print(y)

[[191.  36.  50.]
 [189.  37.  52.]
 [193.  38.  58.]
 [162.  35.  62.]
 [189.  35.  46.]
 [182.  36.  56.]
 [211.  38.  56.]
 [167.  34.  60.]
 [176.  31.  74.]
 [154.  33.  56.]
 [169.  34.  50.]
 [166.  33.  52.]
 [154.  34.  64.]
 [247.  46.  50.]
 [193.  36.  46.]
 [202.  37.  62.]
 [176.  37.  54.]
 [157.  32.  52.]
 [156.  33.  54.]
 [138.  33.  68.]]


In [11]:
y = data.target
y = y[:, 1]
print(y)
x = df.to_numpy()


[36. 37. 38. 35. 35. 36. 38. 34. 31. 33. 34. 33. 34. 46. 36. 37. 37. 32.
 33. 33.]


In [20]:
# y = a0 + a1*x1 + a2*x2 + a3*x3
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=4)

In [21]:
model = LinearRegression().fit(x_train, y_train)
print(model.intercept_)
print(model.coef_)

38.935656643997255
[-0.14675609 -0.02775666  0.02099938]


The Scoring Function gives us an $R^2$ value for our model with the given parameters that it was trained with.

In [22]:
# R2 - 1 - SSres/SStot
print(model.score(x_train, y_train))
print(model.score(x_test, y_test))

0.6383894255040209
0.3712452375606098


In [26]:
import statsmodels.api as sm
est = sm.OLS(y, x).fit()
est.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                      y   R-squared (uncentered):                   0.805
Model:                            OLS   Adj. R-squared (uncentered):              0.771
Method:                 Least Squares   F-statistic:                              23.45
Date:                Mon, 22 Mar 2021   Prob (F-statistic):                    2.83e-06
Time:                        14:36:21   Log-Likelihood:                         -83.423
No. Observations:                  20   AIC:                                      172.8
Df Residuals:                      17   BIC:                                      175.8
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1             0.2759      1.024      0.269      0.791      -1.885       2.436
x2             0.1919      0.084      2.278      0.036       0.014       0.370
x3            -0.0154      0.102     -0.151      0.882      -0.231       0.200
==============================================================================
Omnibus:                        0.745   Durbin-Watson:                   1.584
Prob(Omnibus):                  0.689   Jarque-Bera (JB):                0.683
Skew:                           0.110   Prob(JB):                        0.711
Kurtosis:                       2.122   Cond. No.                         47.8
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

#Validation Metrics

## F-Stat

The F-test is a test where the null hypothesis that all the coefficients are 0 (eg. your model is no better than the mean). Of interest generally is the **p-value** on that test, which should always be be 0.00 -- otherwise your model has big problems.

## Log likelihood

The Log likelihood (LL) comes from the model's [likelihood function](https://en.wikipedia.org/wiki/Likelihood_function). Log-likelihood is all your dataset run through the pdf of the likelihood (normal distribution for OLS), and then they are summed together, and then taking the log of this sum.

So it's measure of **model loss** in the sense that it's a difference between prediction likelihood and reality.

The only real interpretation for log-likelihood is, higher is better.

Log-likelihood values cannot be used alone as an index of model fit because they are a function of sample size but can be used to compare the fit of different coefficients.

## AIC and BIC

The [Akaike information criterion (AIC)](https://en.wikipedia.org/wiki/Akaike_information_criterion) is a **model selection criterion**. It's used to compare different models on the same dataset to compare them.

The equation is $AIC = 2k - 2ln(\hat{L})$ where $k$ is the number of coefficients and $L$ is the model likelihood stat. 

Generally you will compare models on the same dataset and pick the one with the smallest value, this is to penalize models which could be **overfitting** to the data.

The formula for the Bayesian information criterion (BIC) is similar to the formula for AIC, but with a different penalty for the number of parameters. With AIC the penalty is $2k$, whereas with BIC the penalty is $ln(n)k$.

## Durbin Watson

The [Durbin-Watson](https://en.wikipedia.org/wiki/Durbin%E2%80%93Watson_statistic) test checks if the errors in your model have **autocorrelation** (which would imply heteroscedasticity). 

So it's a homoscedasticity test.

##  Skew

Skew is a statistic check for equality of dispersion in your model's error term. A distribution can have right or left-skew, but either way this disperses away from normality of errors, invalidating coefficients and standard errors.

##  Omnibus

A test of the skewness and kurtosis of the residual 

We hope to see a value close to zero which would indicate a normal distribution.

The **Prob (Omnibus)** performs a statistical test indicating the probability that the residuals are normally distributed. We hope to see something close to 1 here. 

##  Kurtosis

A measure of "peakiness", or curvature of the data. Higher peaks lead to greater Kurtosis. Greater Kurtosis can be interpreted as a tighter clustering of residuals around zero, implying a better model with few outliers.

## Jarque-Bera (JB)

Like the Omnibus test in that it tests both skew and kurtosis. We hope to see in this test a confirmation of the Omnibus test.

## Condition Number

This test measures the sensitivity of a function's output as compared to its input.

When we have severe multicollinearity, we can expect much higher fluctuations to small changes in the data, hence, we hope to see a relatively small number, something below 30.

When the $X^-2$ matrix blows up, the condition number will raise a warning.